# `fp8` Weight and Activation Quantization

`llmcompressor` supports quantizing weights and activations to `fp8` for memory savings and inference acceleration with `vllm`
Note: `fp8` compuation is supported on Nvidia GPUs with compute capability > 8.9 (Ada Lovelace, Hopper).

## Installation

Installing llmcompressor may take a minute, depending on the bandwith available

In [ ]:
!pip install -q llmcompressor

In [ ]:
!pip list | grep llmcompressor


### Other dependencies

The next command may show the next ERROR that can be dismiss:
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmcompressor 0.5.0 requires transformers<4.50,>4.0, but you have transformers 4.51.3 which is incompatible.

In [ ]:
!pip install -q -U accelerate vllm boto3

## Quantize the model

There are 4 steps:
1. Load model
2. Apply quantization
3. Evaluate accuracy in vLLM
4. Upload model to S3 (MinIO)

### Load model

Load the model using AutoModelForCausalLM for handling quantized saving and loading.

The model can be loaded from HuggingFace using something like the next:
```python
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_ID = "ibm-granite/granite-3.2-2b-instruct"
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, device_map="auto", torch_dtype="auto",
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
```

But to save time on the lab, we have prepared a MinIO bucket that already contains the model. 

As this workbench was created with a dataconnection attached, the required env vars for accessing the MinIO S3 bucket are defined

NOTE: If you already executed this for the previous ipynb, the model is already downloaded and you can skip this (downloading) step

In [ ]:
import os
import errno
from boto3 import client

MODEL_NAME = "base_model"
MODEL_DOWNLOAD_PATH = "/opt/app-root/src/base_model"

s3_endpoint_url = os.environ["AWS_S3_ENDPOINT"]
s3_access_key = os.environ["AWS_ACCESS_KEY_ID"]
s3_secret_key = os.environ["AWS_SECRET_ACCESS_KEY"]
s3_bucket_name = os.environ["AWS_S3_BUCKET"]

s3_client = client(
    's3', endpoint_url=s3_endpoint_url, aws_access_key_id=s3_access_key,
    aws_secret_access_key=s3_secret_key, verify=False)

# list all objects in the folder
objects = s3_client.list_objects(Bucket=s3_bucket_name, Prefix=MODEL_NAME)

# download each object in the folder
for object in objects['Contents']:
    file_name = object['Key']
    local_file_name = os.path.join(MODEL_DOWNLOAD_PATH, file_name.replace(MODEL_NAME, '')[1:])
    if not os.path.exists(os.path.dirname(local_file_name)):
        try:
            os.makedirs(os.path.dirname(local_file_name))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                print("Error downloading model")
                raise
    s3_client.download_file(s3_bucket_name, file_name, local_file_name)

print('Model downloaded successfully from S3.')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_ID = "/opt/app-root/src/base_model"
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, device_map="auto", torch_dtype="auto",
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

### Apply quantization

For `fp8` quantization, we can recover accuracy with simple PTQ quantization.

We recommend targeting all `Linear` layers using the `FP8_DYNAMIC` scheme, which uses:
- Static, per-channel quantization on the weights
- Dynamic, per-token quantization on the activations

Since simple PTQ does not require data for weight quantization and the activations are quantized dynamically, we do not need any calibration data for this quantization flow.

In [ ]:
from llmcompressor.modifiers.quantization import QuantizationModifier
from llmcompressor.transformers import oneshot

# Configure the simple PTQ quantization
recipe = QuantizationModifier(
  targets="Linear", scheme="FP8_DYNAMIC", ignore=["lm_head"])

# Apply the quantization algorithm.
oneshot(model=model, recipe=recipe)

# Save to disk compressed.
SAVE_DIR = MODEL_ID.split("/")[-1] + "-FP8-Dynamic"
model.save_pretrained(SAVE_DIR, save_compressed=True)
tokenizer.save_pretrained(SAVE_DIR)

### Evaluate accuracy in vLLM

We can evaluate accuracy with lm_eval

##### Check GPU memory leftovers:

In [ ]:
!nvidia-smi

IMPORTANT: 🚨 After quantizing the model the GPU memory may not be freed (see the above output). You need to **restart the kernel** before evaluating the model to ensure you have enough GPU RAM available.

#### Install lm_eval

In [ ]:
!pip install -q lm_eval==v0.4.3

#### Run the evaluation

Run the following to test accuracy on GSM-8K:

In [ ]:
MODEL_ID = "/opt/app-root/src/base_model"
SAVE_DIR = MODEL_ID.split("/")[-1] + "-FP8-Dynamic"

!lm_eval --model vllm \
  --model_args pretrained=$SAVE_DIR,add_bos_token=true,max_model_len=6144 \
  --trust_remote_code \
  --tasks gsm8k \
  --num_fewshot 5 \
  --limit 250 \
  --batch_size 'auto'

If powerfull GPU(s), you could also run the OpenLLM with the following:
```bash
!lm_eval \
  --model vllm \
  --model_args pretrained=$SAVE_DIR,dtype=auto,add_bos_token=True,max_model_len=4096,tensor_parallel_size=1,gpu_memory_utilization=0.8,enable_chunked_prefill=True \
  --trust_remote_code \
  --tasks openllm \
  --write_out \
  --batch_size auto \
  --output_path output_dir \
  --show_config
```

### Upload Optimized Model to MinIO

In [ ]:
import os
from boto3 import client

MODEL_ID = "/opt/app-root/src/base_model"
OPTIMIZED_MODEL_DIR = MODEL_ID.split("/")[-1] + "-FP8-Dynamic"
S3_PATH = "granite-fp8"

print('Starting results upload.')
s3_endpoint_url = os.environ["AWS_S3_ENDPOINT"]
s3_access_key = os.environ["AWS_ACCESS_KEY_ID"]
s3_secret_key = os.environ["AWS_SECRET_ACCESS_KEY"]
s3_bucket_name = os.environ["AWS_S3_BUCKET"]

print(f'Uploading predictions to bucket {s3_bucket_name} '
        f'to S3 storage at {s3_endpoint_url}')

s3_client = client(
    's3', endpoint_url=s3_endpoint_url, aws_access_key_id=s3_access_key,
    aws_secret_access_key=s3_secret_key, verify=False
)

# Walk through the local folder and upload files
for root, dirs, files in os.walk(OPTIMIZED_MODEL_DIR):
    for file in files:
        local_file_path = os.path.join(root, file)
        s3_file_path = os.path.join(S3_PATH, local_file_path[len(OPTIMIZED_MODEL_DIR)+1:])
        s3_client.upload_file(local_file_path, s3_bucket_name, s3_file_path)
        print(f'Uploaded {local_file_path}')

print('Finished uploading results.')